In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
 
################################################
# PLUS 공통 OBJECT
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
 
 
################################################
# PLUS 실행 기본 체크 함수
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # # 주문 관련 초기화 - 계좌 관련 코드가 있을 때만 사용
    # if (g_objCpTrade.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
 
    return True
 

In [2]:
 
class CpMarketEye:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        self.RpFiledIndex = 0
 
 
    def Request(self, codes, dataInfo):
        # 0: 종목코드 4: 현재가 20: 상장주식수 67: PER 71:자본금 72: 액면가
        rqField = [0, 4, 20, 67, 71, 72]  # 요청 필드
 
        self.objRq.SetInputValue(0, rqField)  # 요청 필드
        self.objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        self.objRq.BlockRequest()
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        print("통신상태", rqStatus, self.objRq.GetDibMsg1())
        if rqStatus != 0:
            return False
 
        cnt = self.objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            cur = self.objRq.GetDataValue(1, i)  # 종가
            listedStock = self.objRq.GetDataValue(2, i)  # 상장주식수
            per = self.objRq.GetDataValue(3, i)  # per
            base = self.objRq.GetDataValue(4, i)  # base
            face = self.objRq.GetDataValue(5, i)  # 액면가
 
            maketAmt = listedStock * cur
            if g_objCodeMgr.IsBigListingStock(code) :
                maketAmt *= 1000
                listedStock *= 1000
                print(code, maketAmt)
 
            # key(종목코드) = tuple(상장주식수, 시가총액)
            dataInfo[code] = (listedStock, maketAmt, per, base, face)
 
        return True
 
class CMarketTotal():
    def __init__(self):
        self.dataInfo = {}
 
 
    def GetAllMarketTotal(self):
        codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
        codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
        allcodelist = codeList + codeList2
        allcodelist = ("U001",) + codeList + codeList2
        print('전 종목 코드 %d, 거래소 %d, 코스닥 %d' % (len(allcodelist), len(codeList), len(codeList2)))
 
        objMarket = CpMarketEye()
        rqCodeList = []
        for i, code in enumerate(allcodelist):
            rqCodeList.append(code)
            if len(rqCodeList) == 200:
                objMarket.Request(rqCodeList, self.dataInfo)
                rqCodeList = []
                continue
        # end of for
 
        if len(rqCodeList) > 0:
            objMarket.Request(rqCodeList, self.dataInfo)
 
    def PrintMarketTotal(self):
 
        # 시가총액 순으로 소팅
        data2 = sorted(self.dataInfo.items(), key=lambda x: x[1][1], reverse=True)
 
        print('전종목 시가총액 순 조회 (%d 종목)' % (len(data2)))
        for item in data2:
            name = g_objCodeMgr.CodeToName(item[0])
            listed = item[1][0]
            markettot = item[1][1]
            print('%s 상장주식수: %s, 시가총액 %s' %(name, format(listed, ','), format(markettot, ',')))
 

In [3]:
objMarketTotal = CMarketTotal()
objMarketTotal.GetAllMarketTotal()
objMarketTotal.PrintMarketTotal()

전 종목 코드 2974, 거래소 1559, 코스닥 1414
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
A005930 297892121800000
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
전종목 시가총액 순 조회 (2974 종목)
삼성전자 상장주식수: 5,969,782,000, 시가총액 297,892,121,800,000
SK하이닉스 상장주식수: 728,002,365, 시가총액 59,841,794,403,000
삼성바이오로직스 상장주식수: 66,165,000, 시가총액 39,566,670,000,000
삼성전자우 상장주식수: 822,886,700, 시가총액 35,301,839,430,000
NAVER 상장주식수: 164,263,395, 시가총액 34,905,971,437,500
셀트리온 상장주식수: 134,706,932, 시가총액 28,557,869,584,000
LG화학 상장주식수: 70,592,343, 시가총액 25,342,651,137,000
LG생활건강 상장주식수

본느 상장주식수: 30,699,596, 시가총액 90,256,812,240
퍼스텍 상장주식수: 46,764,840, 시가총액 90,256,141,200
알파홀딩스 상장주식수: 17,747,196, 시가총액 90,155,755,680
기신정기 상장주식수: 29,200,000, 시가총액 90,082,000,000
마이크로텍 상장주식수: 39,896,331, 시가총액 89,966,226,405
솔트웍스 상장주식수: 13,204,819, 시가총액 89,924,817,390
바이오스마트 상장주식수: 20,014,594, 시가총액 89,865,527,060
코드네이처 상장주식수: 30,151,608, 시가총액 89,851,791,840
대원산업 상장주식수: 20,037,600, 시가총액 89,668,260,000
고려신용정보 상장주식수: 14,300,000, 시가총액 89,661,000,000
우리들제약 상장주식수: 13,555,220, 시가총액 89,600,004,200
이랜텍 상장주식수: 21,985,432, 시가총액 89,480,708,240
가온미디어 상장주식수: 15,074,293, 시가총액 89,390,557,490
디케이티 상장주식수: 8,231,333, 시가총액 89,309,963,050
호전실업 상장주식수: 8,000,000, 시가총액 89,200,000,000
신흥 상장주식수: 9,600,000, 시가총액 89,184,000,000
엘엠에스 상장주식수: 8,895,755, 시가총액 88,957,550,000
무림페이퍼 상장주식수: 41,609,310, 시가총액 88,835,876,850
녹원씨엔아이 상장주식수: 16,138,934, 시가총액 88,764,137,000
한컴위드 상장주식수: 28,217,081, 시가총액 88,742,719,745
한국제지 상장주식수: 5,004,949, 시가총액 88,587,597,300
현대코퍼레이션홀딩스 상장주식수: 9,100,836, 시가총액 88,460,125,920
EMW 상장주식수: 31,784,197, 시가총

TIGER 지주회사 상장주식수: 550,000, 시가총액 5,192,000,000
KINDEX 배당성장 상장주식수: 200,000, 시가총액 5,185,000,000
ARIRANG KRX300헬스케어 상장주식수: 500,000, 시가총액 5,152,500,000
DB하이텍1우 상장주식수: 112,316, 시가총액 5,127,225,400
하나니켈2호 상장주식수: 3,284,000, 시가총액 5,106,620,000
TIGER 대형성장 상장주식수: 600,000, 시가총액 5,097,000,000
KODEX 구리선물(H) 상장주식수: 1,100,000, 시가총액 5,076,500,000
TIGER MSCI KOREA ESG리더스 상장주식수: 690,000, 시가총액 5,061,150,000
KODEX MSCI모멘텀 상장주식수: 600,000, 시가총액 5,058,000,000
KODEX 국채선물3년인버스 상장주식수: 102,000, 시가총액 5,049,510,000
KODEX 가치투자 상장주식수: 600,000, 시가총액 4,983,000,000
KINDEX 국채선물3년인버스 상장주식수: 50,000, 시가총액 4,943,000,000
KINDEX 스마트퀄리티 상장주식수: 400,000, 시가총액 4,938,000,000
금강공업우 상장주식수: 589,875, 시가총액 4,937,253,750
KBSTAR 우량업종 상장주식수: 600,000, 시가총액 4,920,000,000
KBSTAR 게임테마 상장주식수: 500,000, 시가총액 4,890,000,000
KBSTAR 200커뮤니케이션서비스 상장주식수: 400,000, 시가총액 4,842,000,000
KINDEX 한류 상장주식수: 800,000, 시가총액 4,824,000,000
TIGER 200 경기소비재 상장주식수: 440,000, 시가총액 4,787,200,000
TIGER K게임 상장주식수: 550,000, 시가총액 4,771,250,000
ARIRANG 코스닥150 상장주식수: 450,000, 시가

In [4]:
data2 = sorted(objMarketTotal.dataInfo.items(), key=lambda x: x[1][1], reverse=True)

In [5]:
data2[0]

('A005930', (5969782000, 297892121800000, 15.760000228881836, 778046, 100))

In [6]:
MarketTotal = []

for item in data2:
    name = g_objCodeMgr.CodeToName(item[0])
    listed = item[1][0]
    markettot = item[1][1]
    per = item[1][2]
    eps = item[1][3]
    face = item[1][4]
    
    MarketTotal.append([name, item[0], listed, markettot, per, eps, face])

In [7]:
import pandas as pd

In [8]:
format(1e+10, ",")

'10,000,000,000.0'

In [9]:
format(1e+12, ",")

'1,000,000,000,000.0'

In [10]:
pdMarketTotal = pd.DataFrame(MarketTotal)

In [11]:
len(pdMarketTotal)

2974

In [12]:
pdMarketTotal.to_csv("../data/KRX_LIST.csv", index=None)

In [13]:
pdMarketTotal.columns = "Name,Code,Stock,capitalization,per,base,face".split(",")

In [14]:
pdMarketTotal.loc[pdMarketTotal.Code == "U001"]

,Name,Code,Stock,capitalization,per,base,face
2973,종합주가지수,U001,0,0.0,0.0,0,0


In [15]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 200선물인버스2X"]

,Name,Code,Stock,capitalization,per,base,face
102,KODEX 200선물인버스2X,A252670,320900000,2.056969e+12,0.0,0,0


In [16]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 미국S&P500선물(H)"]

,Name,Code,Stock,capitalization,per,base,face
1471,KODEX 미국S&P500선물(H),A219480,5750000,8.052875e+10,0.0,0,0


In [17]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 미국S&P500선물(H)"]

,Name,Code,Stock,capitalization,per,base,face
1471,KODEX 미국S&P500선물(H),A219480,5750000,8.052875e+10,0.0,0,0


In [18]:
pdMarketTotal.loc[pdMarketTotal.Code == "A950140"]

,Name,Code,Stock,capitalization,per,base,face
1234,잉글우드랩,A950140,19867866,1.043063e+11,46.049999,0,0


In [19]:
pdMarketTotal.loc[pdMarketTotal.Code == "A900140"]

,Name,Code,Stock,capitalization,per,base,face
623,엘브이엠씨홀딩스,A900140,51120439,2.510014e+11,0.0,12,0


In [20]:
pdMarketTotal.loc[pdMarketTotal.Code == "A005930"]

,Name,Code,Stock,capitalization,per,base,face
0,삼성전자,A005930,5969782000,2.978921e+14,15.76,778046,100


In [21]:
pdMarketTotal.loc[pdMarketTotal.Code == "A000660"]

,Name,Code,Stock,capitalization,per,base,face
1,SK하이닉스,A000660,728002365,5.984179e+13,29.709999,3657652,5000


In [22]:
# pdMarketTotal_not_etf = pdMarketTotal.loc[(
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KODEX") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("TIGER") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KBSTAR") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("HANARO") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ARIRANG") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KINDEX") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KOSEF") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("SMART") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("TREX") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("OTM") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ATM") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ETN") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ETF") < 0)) 
# )]

In [23]:
pdMarketTotal = pdMarketTotal[pdMarketTotal.Code.apply(lambda x: not x.endswith("K"))]
pdMarketTotal = pdMarketTotal[pdMarketTotal.Code.apply(lambda x: not x.startswith("Q"))]

In [24]:
KOSPI = pdMarketTotal.loc[(
    (pdMarketTotal.Stock == 0)
), "Name,Code,Stock,capitalization".split(",")]

In [25]:
KOSPI["type"] = "index"

In [26]:
STOCK = pdMarketTotal.loc[(
    ((pdMarketTotal.face > 0) & 
     (pdMarketTotal.base > 0) & 
     (pdMarketTotal.per > 0) & 
     (pdMarketTotal.Stock > 1e+7) &
     (pdMarketTotal.capitalization > 1e+12))
), "Name,Code,Stock,capitalization".split(",")]

In [27]:
STOCK["type"] = "stock"

In [28]:
ETF = pdMarketTotal.loc[(
    ((pdMarketTotal.face == 0) & 
     (pdMarketTotal.base == 0) & 
     (pdMarketTotal.per == 0) & 
     (pdMarketTotal.capitalization > 1e+10))
), "Name,Code,Stock,capitalization".split(",")]

In [29]:
ETF["type"] = 'etf'

In [30]:
concat = pd.concat([KOSPI, STOCK, ETF])

In [31]:
concat

,Name,Code,Stock,capitalization,type
2973,종합주가지수,U001,0,0.000000e+00,index
0,삼성전자,A005930,5969782000,2.978921e+14,stock
1,SK하이닉스,A000660,728002365,5.984179e+13,stock
2,삼성바이오로직스,A207940,66165000,3.956667e+13,stock
4,NAVER,A035420,164263395,3.490597e+13,stock
...,...,...,...,...,...
2611,KODEX 헬스케어,A266420,700000,1.038100e+10,etf
2615,HANARO e커머스,A322400,950000,1.026950e+10,etf
2618,KODEX TRF5050,A329660,1000000,1.023000e+10,etf
2619,KODEX TRF7030,A329670,1000000,1.021000e+10,etf


In [40]:
concat.to_csv("../data/SELECT_LIST.csv", index=None)